In [10]:
import pandas as pd
import re

In [59]:
pattern = re.compile(r"\[(.*?)\]")

wake_up = []
fall_asleep = []
begin_shift = []

with open("input.txt") as f:
    guards = []
    for l in f:
        a, b = pattern.match(l).span()
        date = "2018"+l[a+1: b-1][4:]
        event = l[b+1:].strip()
        if event == "wakes up":
            wake_up.append((date, 1))
        elif event == "falls asleep":
            fall_asleep.append((date, 1))
        else:
            guard_id = int(event.split()[1][1:])
            begin_shift.append((date, guard_id))

In [133]:
df = pd.concat([pd.DataFrame(begin_shift, columns=["date", "guard"]),
               pd.DataFrame(fall_asleep, columns=["date", "fall_asleep"]),
               pd.DataFrame(wake_up, columns=["date", "wake_up"])], sort=False)

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").set_index("date")
df["guard"] = df["guard"].ffill()
df = df.query("index.dt.hour == 0")

In [149]:
results = []
for guard_id, grp in df.groupby("guard", sort=False):
    grp = grp.reindex(pd.date_range(grp.index[0], grp.index[~0], freq="60S"))
    grp = grp.fillna(0)
    grp["sleeping"] = grp["fall_asleep"].cumsum()-grp["wake_up"].cumsum()
    grp["min"] = grp.index.minute
    r = grp.groupby("min")["sleeping"].sum().sort_values(ascending=False)
    print(r)
    results.append((r.iloc[0], r.index[0], guard_id))

min
44    10.0
48    10.0
47    10.0
45     9.0
40     9.0
41     9.0
46     9.0
42     8.0
43     8.0
50     8.0
49     8.0
28     7.0
37     7.0
39     7.0
29     7.0
51     7.0
54     7.0
30     6.0
52     6.0
53     6.0
38     6.0
36     6.0
35     6.0
26     6.0
27     6.0
31     6.0
25     5.0
22     5.0
21     5.0
32     5.0
33     5.0
17     4.0
20     4.0
12     4.0
13     4.0
14     4.0
15     4.0
16     4.0
19     4.0
18     4.0
56     4.0
23     4.0
24     4.0
55     4.0
34     4.0
11     3.0
10     2.0
57     1.0
7      1.0
9      1.0
8      1.0
2      0.0
1      0.0
4      0.0
3      0.0
59     0.0
5      0.0
6      0.0
58     0.0
0      0.0
Name: sleeping, dtype: float64
min
36    9.0
35    9.0
32    8.0
37    8.0
33    8.0
30    7.0
34    7.0
38    7.0
31    7.0
46    6.0
45    6.0
41    6.0
40    6.0
39    6.0
28    5.0
27    5.0
26    5.0
25    5.0
24    5.0
23    5.0
18    5.0
29    5.0
42    5.0
44    5.0
47    5.0
48    5.0
49    5.0
43    5.0
56    4.0
17    4.0
5

min
34    12.0
35    12.0
36    11.0
29    10.0
31    10.0
32    10.0
33    10.0
37    10.0
38    10.0
39    10.0
41     9.0
40     9.0
30     9.0
28     8.0
44     8.0
43     8.0
42     8.0
25     8.0
48     8.0
26     8.0
50     8.0
51     8.0
52     8.0
16     8.0
17     8.0
46     7.0
27     7.0
53     7.0
49     7.0
47     7.0
45     7.0
19     6.0
20     6.0
21     6.0
18     6.0
22     6.0
24     6.0
23     6.0
55     6.0
56     5.0
54     5.0
15     5.0
14     4.0
11     4.0
13     4.0
12     4.0
10     3.0
9      3.0
8      3.0
7      3.0
6      3.0
5      3.0
57     2.0
58     2.0
1      1.0
4      1.0
3      1.0
2      1.0
59     0.0
0      0.0
Name: sleeping, dtype: float64
min
37    11.0
38    10.0
36    10.0
41     9.0
44     9.0
45     9.0
35     9.0
39     9.0
46     9.0
47     8.0
42     8.0
43     8.0
40     8.0
48     7.0
34     7.0
33     7.0
32     7.0
31     7.0
28     6.0
29     6.0
30     6.0
49     6.0
27     5.0
24     5.0
25     5.0
26     5.0
50     5.0
53  

In [150]:
print(results)

[(10.0, 44, 433.0), (9.0, 36, 677.0), (8.0, 35, 277.0), (12.0, 24, 109.0), (9.0, 47, 239.0), (15.0, 21, 1097.0), (14.0, 44, 2251.0), (6.0, 23, 1171.0), (10.0, 43, 2887.0), (11.0, 38, 103.0), (13.0, 43, 1447.0), (13.0, 44, 2593.0), (13.0, 33, 2423.0), (12.0, 34, 1871.0), (11.0, 37, 739.0), (13.0, 49, 727.0), (12.0, 31, 2689.0), (0.0, 59, 281.0), (0.0, 0, 3121.0), (0.0, 3, 821.0), (5.0, 17, 1249.0), (5.0, 53, 1709.0), (4.0, 43, 1051.0)]


In [141]:
max(results)

(15.0, 21, 1097.0)

In [142]:
1097*21

23037